In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
# import torch.utils.data.dataloader as dataloader
# import torch.utils.data.DataLoader as DataLoader
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from tqdm import tqdm, trange

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
# Hyper-parameters 
num_epochs = 5
batch_size = 4
learning_rate = 0.001

In [ ]:
# dataset has PILImage images of range [0, 1]. 
# We transform them to Tensors of normalized range [-1, 1]
transform  = transforms.Compose([transforms.ToTensor(),
                            transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [ ]:
# CIFAR10: 60000 32x32 color images in 10 classes, with 6000 images per class
train_dataset = datasets.CIFAR10(root='./data',train=True,transform=transform ,download=True)
test_dataset = datasets.CIFAR10(root='./data',train=False,transform=transform ,download=False)

train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=True)


In [ ]:
train_dataset.classes

In [ ]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
def imshow(img):
    img = img * 0.5 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)))
    plt.show()

In [ ]:
images, labels = next(iter(train_loader))
imshow(torchvision.utils.make_grid(images))
[classes[x] for x in labels]

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool1(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) 
        return x

In [ ]:
model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr= learning_rate)

In [ ]:
num_iters = len(train_dataset) / batch_size
with tqdm(total=num_epochs*len(train_loader)) as pbar:
    for epoch in range(num_epochs):
        for i,(images, labels) in enumerate(train_loader):
            pbar.update(1)
            # to gpu
            images = images.to(device)
            labels = labels.to(device)

            # origin shape: [4, 3, 32, 32] = 4, 3, 1024
            # input_layer: 3 input channels, 6 output channels, 5 kernel size

            # forward pass
            output = model(images)
            loss = criterion(output, labels)

            # backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % 2500 ==0:
                # pbar.set_postfix(epoch=(epoch+1)/num_epochs, iter=(i+1)/num_iters, loss=f'{loss.item():.2f}')
                print(f'eopch : {epoch+1}/{num_epochs}, iter : {i+1}/{num_iters}, loss : {loss.item():.2f}')

In [ ]:
len(train_loader)

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

In [ ]:
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

In [ ]:
from torchsummary import summary

In [ ]:
train_dataset[0][0].shape

In [ ]:
summary(model, (3,32,32))

In [ ]:
len(train_dataset)

In [ ]:
images, labels = next(iter(train_loader))
imshow(images[0])
img = images[0]

In [ ]:
# cnn feature map size example



conv1 = nn.Conv2d(3, 6, 5)
pool1 = nn.MaxPool2d(2, 2)
conv2 = nn.Conv2d(6, 16, 5)

In [ ]:
images.shape

In [ ]:
x = conv1(images)
x.shape

In [ ]:
x = pool1(x)
x.shape

In [ ]:
x = conv2(x)
x.shape